<h1 align=center><font size = 5 >Capstone Project | Explore Segmenting and cluster the neighborhood of | Gujarat | India </font></h1>

## Table of content

* 1. Importing all required Libraries
* 2. Scrap Data from wikipedia page into a DataFrame
* 3. Get the Geographical Coordinates
* 4. Create map of All Enifneering college in Gujarat using Latitude and longitude
* 5. Use the Foursquare API to explore the neighborhoods
* 6. Analyze Each Neighborhood
* 7. Examine clusters

**1.Importing all required Libraries**

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Required Libraries imported.")

**2.Scrap Data from wikipedia page into a DataFrame**

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Engineering_colleges_in_Gujarat").text

In [3]:
soup = BeautifulSoup(data, 'html.parser')

In [4]:
neighborhoodList = []

In [5]:
for row in soup.find_all("div", class_ = "mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df.head(50)

,Neighborhood
0,A. D. Patel Institute of Technology
1,G. K. Bharad Institute of Engineering
2,Birla Vishvakarma Mahavidyalaya
3,C. K. Pithawala College of Engineering and Tec...
4,C. U. Shah College of Engineering and Technolo...
5,Charotar University of Science and Technology
6,Dharamsinh Desai University
7,Dhirubhai Ambani Institute of Information and ...
8,"Dr. Jivraj Mehta Institute of Technology, Anand"
9,G. H. Patel College of Engineering and Technology


In [7]:
df.shape

(30, 1)

**3. Get the Geographical Coordinates**

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Gujarat, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [10]:
coords

[[22.521660000000054, 72.91652000000005],
 [22.236780000000067, 70.90140000000008],
 [22.552990000000023, 72.92377000000005],
 [21.131380000000036, 72.71913000000006],
 [23.03869000000003, 72.55623000000008],
 [22.597750000000076, 72.82149000000004],
 [22.67948000000007, 72.87972000000008],
 [23.188550000000077, 72.62986000000006],
 [22.521660000000054, 72.91652000000005],
 [23.20455000000004, 73.60179000000005],
 [21.759870000000035, 72.13276000000008],
 [23.236240000000066, 69.65855000000005],
 [23.25967000000003, 72.64887000000004],
 [23.809160000000077, 72.10329000000007],
 [23.084150000000022, 72.75688000000008],
 [23.004400000000032, 72.62013000000007],
 [22.566140000000075, 73.24305000000004],
 [23.03553000000005, 72.54613000000006],
 [22.817540000000065, 70.84656000000007],
 [22.265690000000063, 70.71214000000003],
 [22.892120000000034, 72.59768000000008],
 [21.167650000000037, 72.78518000000008],
 [23.019070000000056, 72.57219000000003],
 [23.03869000000003, 72.55623000000008]

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [13]:
# check the neighborhoods and the coordinates
print(df.shape)
df

(30, 3)


,Neighborhood,Latitude,Longitude
0,A. D. Patel Institute of Technology,22.52166,72.91652
1,G. K. Bharad Institute of Engineering,22.23678,70.90140
2,Birla Vishvakarma Mahavidyalaya,22.55299,72.92377
3,C. K. Pithawala College of Engineering and Tec...,21.13138,72.71913
4,C. U. Shah College of Engineering and Technolo...,23.03869,72.55623
5,Charotar University of Science and Technology,22.59775,72.82149
6,Dharamsinh Desai University,22.67948,72.87972
7,Dhirubhai Ambani Institute of Information and ...,23.18855,72.62986
8,"Dr. Jivraj Mehta Institute of Technology, Anand",22.52166,72.91652
9,G. H. Patel College of Engineering and Technology,23.20455,73.60179


In [14]:
df.to_csv("df.csv", index = False)

In [15]:
# get the coordinates of Kuala Lumpur
address = 'Gujarat, India'

geolocator = Nominatim(user_agent="ny-explore")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Building and structure of Gujarat, India {}, {}.'.format(latitude, longitude))

The Building and structure of Gujarat, India 22.3850051, 71.745261.


**4. Create map of All Enifneering college in Gujarat using Latitude and longitude**

In [16]:
map_Guj = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to map
for lat,lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lng], radius = 6, popup = label,
                        color ='red',fill=True,fill_color = '#3186cc',
                        fill_opacity = 0.7).add_to(map_Guj)

In [17]:
map_Guj

**Save the Map as HTML file**

In [18]:
map_Guj.save('map_Guj.html')

**5. Use the Foursquare API to explore the neighborhoods**

In [19]:
CLIENT_ID='0ZOFFGKXDEMNTGR1IE1TZCCY04OLYB3JP4P1S2LFWIKMEKAG'
CLIENT_SECRET = '5QJHKEMHKC25YJWNPZK1QQGAJSTHFVPYBALZYLJZ2QN5ZNRN'
VERSION = '20180604'

print('Your Cridentials')
print('CLIENT_ID : ', CLIENT_ID)
print('CLIENT_SECRET : ', CLIENT_SECRET)
print('VERSION :', VERSION)

Your Cridentials
CLIENT_ID :  0ZOFFGKXDEMNTGR1IE1TZCCY04OLYB3JP4P1S2LFWIKMEKAG
CLIENT_SECRET :  5QJHKEMHKC25YJWNPZK1QQGAJSTHFVPYBALZYLJZ2QN5ZNRN
VERSION : 20180604


**Now , Let's get the top venues.**

In [20]:
radius = 2000
LIMIT = 100
venues = []

In [21]:
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # Create The API request URL from FourSquare 
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,
        LIMIT
    )
    
    # For get request
    results = requests.get(url).json()["response"]["groups"][0]['items']
    
    # retuen only relevant information for each nearby venue
    
    for venue in results:
        venues.append((
        neighborhood,
            lat,
            lng,
            venue['venue']['name'],
            venue['venue']['location']['lat'],
            venue['venue']['location']['lng'],
            venue['venue']['categories'][0]['name']))

**Convert the venues into a new DataFrame**

In [22]:
venue_df = pd.DataFrame(venues)

**Define the columns names**

In [23]:
venue_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'Venue Name', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category']

In [24]:
venue_df.shape

(517, 7)

In [25]:
venue_df.head(30)

,Neighborhood,Latitude,Longitude,Venue Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,A. D. Patel Institute of Technology,22.52166,72.91652,Hot Spice,22.518518,72.919339,Fast Food Restaurant
1,A. D. Patel Institute of Technology,22.52166,72.91652,"Wedesi Cafe,Anand",22.529139,72.927220,Fast Food Restaurant
2,A. D. Patel Institute of Technology,22.52166,72.91652,Vipra Agency,22.532905,72.923232,Ice Cream Shop
3,A. D. Patel Institute of Technology,22.52166,72.91652,TNT Express,22.531796,72.926127,Business Service
4,G. K. Bharad Institute of Engineering,22.23678,70.90140,Savaliya Farm,22.239081,70.906099,Farm
5,Birla Vishvakarma Mahavidyalaya,22.55299,72.92377,Satyanarayan Icecream Parlour,22.550031,72.931069,Dessert Shop
6,Birla Vishvakarma Mahavidyalaya,22.55299,72.92377,Brewberry's,22.555416,72.923451,Café
7,Birla Vishvakarma Mahavidyalaya,22.55299,72.92377,Garam Kitli,22.546773,72.924697,Café
8,Birla Vishvakarma Mahavidyalaya,22.55299,72.92377,Subway,22.549433,72.928183,Sandwich Place
9,Birla Vishvakarma Mahavidyalaya,22.55299,72.92377,Cheese & chips,22.548605,72.925573,BBQ Joint


**Let's check how many vanues were returned for each neighborhood**

In [26]:
venue_df['Neighborhood'].count()

517

In [27]:
venue_df.groupby(['Neighborhood']).count()

,Latitude,Longitude,Venue Name,Venue_Latitude,Venue_Longitude,Venue_Category
Neighborhood,,,,,,
A. D. Patel Institute of Technology,4,4,4,4,4,4
Birla Vishvakarma Mahavidyalaya,16,16,16,16,16,16
C. K. Pithawala College of Engineering and Technology,3,3,3,3,3,3
"C. U. Shah College of Engineering and Technology, Surendranagar",100,100,100,100,100,100
Charotar University of Science and Technology,1,1,1,1,1,1
Dharamsinh Desai University,5,5,5,5,5,5
Dhirubhai Ambani Institute of Information and Communication Technology,14,14,14,14,14,14
"Dr. Jivraj Mehta Institute of Technology, Anand",4,4,4,4,4,4
G. H. Patel College of Engineering and Technology,3,3,3,3,3,3


**Let's find out how many unique categories .**

In [28]:
print('There are {} uniques categories'.format(len(venue_df['Venue_Category'].unique())))

There are 64 uniques categories


**Print the unique list of categories .**

In [29]:
venue_df['Venue_Category'].unique()[:50]

array(['Fast Food Restaurant', 'Ice Cream Shop', 'Business Service',
       'Farm', 'Dessert Shop', 'Café', 'Sandwich Place', 'BBQ Joint',
       'Food', 'Indian Restaurant', 'Asian Restaurant', 'Shopping Mall',
       'Clothing Store', 'Coffee Shop', 'Hotel', 'Tennis Court',
       'Theme Park', 'Diner', 'Cricket Ground', 'Bookstore',
       'Art Gallery', 'Farmers Market', 'Mexican Restaurant', 'Bakery',
       'Moroccan Restaurant', 'Gym', 'Snack Place', 'Tea Room',
       'Pizza Place', 'Falafel Restaurant',
       'Vegetarian / Vegan Restaurant', 'Arts & Crafts Store', 'Park',
       'Performing Arts Venue', 'Restaurant', 'Multiplex', 'Theater',
       'Breakfast Spot', 'Arcade', 'ATM', 'Train Station', 'Trail',
       'Italian Restaurant', 'Historic Site', 'Gym / Fitness Center',
       'History Museum', 'Tennis Stadium', 'Garden', 'Shoe Store',
       'Electronics Store'], dtype=object)

In [30]:
"Neighborhood" in venue_df['Venue_Category'].unique()

False

**6. Analyze Each Neighborhood**

In [31]:
Gj_onehot = pd.get_dummies(venue_df[['Venue_Category']], prefix = "", prefix_sep = "")


**Add neighborhood columns to the first column**

In [32]:
Gj_onehot['Neighborhood'] = venue_df['Neighborhood']

In [33]:
# Move neighborhood columns to the first column
fixed_columns = [Gj_onehot.columns[-1]] + list(Gj_onehot.columns[:-1])
Gj_onehot = Gj_onehot[fixed_columns]

In [34]:
print(Gj_onehot.shape)

(517, 65)


In [35]:
Gj_onehot.head()

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bookstore,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,Frozen Yogurt Shop,Garden,Gourmet Shop,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Men's Store,Mexican Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Performing Arts Venue,Pizza Place,Resort,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant
0,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,G. K. Bharad Institute of Engineering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Next let's group rows by neighborhood and by taking the mean of the frequency of occurance of each category**

In [36]:
Gj_grouped = Gj_onehot.groupby(["Neighborhood"]).mean().reset_index

In [37]:
Gj_grouped()

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bookstore,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,Frozen Yogurt Shop,Garden,Gourmet Shop,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Men's Store,Mexican Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Performing Arts Venue,Pizza Place,Resort,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant
0,A. D. Patel Institute of Technology,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.500000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,Birla Vishvakarma Mahavidyalaya,0.000000,0.000000,0.00,0.000000,0.062500,0.062500,0.000000,0.000000,0.000000,0.00,0.00,0.125000,0.000000,0.062500,0.062500,0.000000,0.125000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.125000,0.062500,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.125000,0.062500,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.062500,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,C. K. Pithawala College of Engineering and Tec...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.00,0.000000,0.000000,0.000000
3,C. U. Shah College of Engineering and Technolo...,0.000000,0.010000,0.01,0.010000,0.020000,0.000000,0.020000,0.010000,0.010000,0.00,0.00,0.210000,0.000000,0.010000,0.020000,0.020000,0.050000,0.010000,0.00,0.000000,0.01,0.0,0.01,0.120000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.000000,0.030000,0.010000,0.130000,0.000000,0.000000,0.00,0.030000,0.01,0.000000,0.010000,0.000000,0.010000,0.01,0.050000,0.00,0.010000,0.030000,0.000000,0.020000,0.000000,0.030000,0.030000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.020000
4,Charotar University of Science and Technology,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,Dharamsinh Desai University,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.00,0.00000

In [38]:
num_top_venues = 5
for hood in Gj_onehot["Neighborhood"]:
    print("---- "+hood+" ----")
    temp =Gj_onehot[Gj_onehot['Neighborhood'] == hood].T.reset_index()

---- A. D. Patel Institute of Technology ----
---- A. D. Patel Institute of Technology ----
---- A. D. Patel Institute of Technology ----
---- A. D. Patel Institute of Technology ----
---- G. K. Bharad Institute of Engineering ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- Birla Vishvakarma Mahavidyalaya ----
---- C. K. Pithawala College of Engineering and Technology ----
---- C. K. Pithawala College of 

---- Government Engineering College, Bhavnagar ----
---- Government Engineering College, Bhavnagar ----
---- Government Engineering College, Bhuj ----
---- Government Engineering College, Bhuj ----
---- Government Engineering College, Bhuj ----
---- Government Engineering College, Bhuj ----
---- Government Engineering College, Bhuj ----
---- Government Engineering College, Bhuj ----
---- Government Engineering College, Bhuj ----
---- Government Engineering College, Gandhinagar ----
---- Government Engineering College, Gandhinagar ----
---- Government Engineering College, Gandhinagar ----
---- Government Engineering College, Gandhinagar ----
---- Government Engineering College, Gandhinagar ----
---- Government Engineering College, Gandhinagar ----
---- Government Engineering College, Gandhinagar ----
---- Institute of Infrastructure Technology Research and Management ----
---- Institute of Infrastructure Technology Research and Management ----
---- Institute of Infrastructure Technology

---- Sardar Vallabhbhai National Institute of Technology, Surat ----
---- Sardar Vallabhbhai National Institute of Technology, Surat ----
---- Sardar Vallabhbhai National Institute of Technology, Surat ----
---- Sardar Vallabhbhai National Institute of Technology, Surat ----
---- Sardar Vallabhbhai National Institute of Technology, Surat ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- Sardar Vallabhbhai Patel Institute of Technology ----
---- S

---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Sarvajanik College of Engineering and Technology ----
---- Shantilal Shah Engineering College ----
---- Shantilal Shah Engineering College ----
---- Shri Labhubhai Trivedi Institute of Engineering & Technology, Rajkot ----
---- Shri Labhubhai Trivedi Institute of Engineering & Technolog

In [39]:
# set number of cluster
kclusters = 3

Gj_clustering = Gj_onehot[["Fast Food Restaurant"]]

# run k-means clustering
kmeans = KMeans(kclusters, random_state = 0).fit(Gj_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

C:\Users\Admin\anaconda3\lib\site-packages\ipykernel_launcher.py:5: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  """


array([1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 1])

In [40]:
Gj_merged = Gj_onehot.copy()
Gj_merged["Cluster Labels"] = kmeans.labels_
print(Gj_merged.shape)
Gj_merged.head()

(517, 66)


,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bookstore,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,Frozen Yogurt Shop,Garden,Gourmet Shop,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Men's Store,Mexican Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Performing Arts Venue,Pizza Place,Resort,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
0,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,G. K. Bharad Institute of Engineering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [41]:
Gj_merged.rename(columns = {"Neighborhood": "Neighborhood"}, inplace =True)
Gj_merged.head()

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bookstore,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,Frozen Yogurt Shop,Garden,Gourmet Shop,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Men's Store,Mexican Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Performing Arts Venue,Pizza Place,Resort,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
0,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,G. K. Bharad Institute of Engineering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


**7. Examine clusters**

In [42]:
Gj_merged[Gj_merged["Cluster Labels"] == 0]

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bookstore,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,Frozen Yogurt Shop,Garden,Gourmet Shop,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Men's Store,Mexican Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Performing Arts Venue,Pizza Place,Resort,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels


In [43]:
Gj_merged[Gj_merged["Cluster Labels"] == 1]

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bookstore,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,Frozen Yogurt Shop,Garden,Gourmet Shop,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Men's Store,Mexican Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Performing Arts Venue,Pizza Place,Resort,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
0,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
10,Birla Vishvakarma Mahavidyalaya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
14,Birla Vishvakarma Mahavidyalaya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
42,C. U. Shah College of Engineering and Technolo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
49,C. U. Shah College of Engineering and Technolo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
56,C. U. Shah College of Engineering and Technolo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
76,C. U. Shah College of Engineering and Technolo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
85,C. U. Shah College of Engineering and Technolo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
89,C. U. Shah College of Engineering and Technolo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [44]:
Gj_merged[Gj_merged["Cluster Labels"] == 2]

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bookstore,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,Frozen Yogurt Shop,Garden,Gourmet Shop,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Men's Store,Mexican Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Performing Arts Venue,Pizza Place,Resort,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
2,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,A. D. Patel Institute of Technology,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,G. K. Bharad Institute of Engineering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
5,Birla Vishvakarma Mahavidyalaya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
6,Birla Vishvakarma Mahavidyalaya,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
7,Birla Vishvakarma Mahavidyalaya,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
8,Birla Vishvakarma Mahavidyalaya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2
9,Birla Vishvakarma Mahavidyalaya,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
11,Birla Vishvakarma Mahavidyalaya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
12,Birla Vishvakarma Mahavidyalaya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [45]:
Gj_merged[Gj_merged["Cluster Labels"] == 3]

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bookstore,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cricket Ground,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Food,Food Court,Food Truck,Frozen Yogurt Shop,Garden,Gourmet Shop,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Men's Store,Mexican Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,North Indian Restaurant,Park,Performing Arts Venue,Pizza Place,Resort,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
